In [1]:
import numpy as np
import scipy.io.wavfile as wav
from librosa import load
from librosa.feature import mfcc

directory="C:/"
folder="Project"
file="StarWars60.wav"
sig, rate = load(directory+folder+"/"+file)
print(rate)
wav.read(directory+folder+"/"+file)
# computing MFCC features using librosa
mfcc_feat = mfcc(y=sig, sr=rate, n_fft=512)
print(mfcc_feat.shape)
# Mean and Covariance matrix to assist with distance computation
covariance = np.cov(mfcc_feat)
mean_matrix = mfcc_feat.mean(axis=1, dtype=np.float64)

feature = (mean_matrix, covariance)
print(mean_matrix.shape, covariance.shape, sep='\n')

22050
(20, 2584)
(20,)
(20, 20)


In [2]:
import os
import pickle
import numpy as np
import scipy.io.wavfile as wav
from librosa import load
from librosa.feature import mfcc

def preprocess(directory="C:/Project/", filename="lr.dat"):
    f = open(filename ,'wb')
    i=0
    for folder in os.listdir(directory):
        i += 1
        if i == 11 :
            break 	
        for file in os.listdir(directory+folder):
            print("Working with: " + directory + folder + "/" + file)
            # Reading the audio file
            sig, rate = load(directory+folder+"/"+file, sr=None)
            #wav.read(directory+folder+"/"+file)
            # computing MFCC features using librosa
            mfcc_feat = mfcc(y=sig, sr=rate)
            # Mean and Covariance matrix to assist with distance computation
            covariance = np.cov(mfcc_feat)
            mean_matrix = mfcc_feat.mean(axis=1, dtype=np.float64)
            
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
    f.close()

def loadDataset(filename="lr.dat"):
    if not os.path.exists(filename):
        preprocess(filename=filename)
    data, labels = [], []
    with open(filename,'rb') as f:
        while True:
            try:
                temp = pickle.load(f)
                data.append([temp[0]] + list(temp[1]))
                labels.append(temp[-1])
            except EOFError:
                f.close()
                break
    # Conversion to NumPy arrays for easier manipulation
    data, labels = np.array(data), np.array(labels)
    return data, labels

In [3]:
import numpy as np
import operator

# Accuracy from the testing predictions
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range(len(testSet)):
        if testSet[x] == predictions[x]:
            correct += 1
    return 1.0 * correct / len(testSet)

# A custom distance function for use with k-NN
def distance(instance1, instance2, k):
    mm1 = instance1[0] 
    cm1 = instance1[1:]
    dcm1 = np.linalg.det(cm1)
    mm2 = instance2[0]
    cm2 = instance2[1:]
    dcm2 = np.linalg.det(cm2)
    icm2 = np.linalg.inv(cm2)
    dmm = mm2 - mm1
    
    # 
    distance = np.trace(np.dot(icm2, cm1))
    # Mahalanobis distance between the two instances
    distance += np.sqrt(np.dot(np.dot((dmm).transpose(), icm2), dmm)) 
    # Difference in Differential entropy between instances
    # (measured indirectly as a property of Covariance matrices)
    distance += np.log(dcm2) - np.log(dcm1)
    distance -= k
    return distance

# A function which finds k neighbours of the given instance in the training set
def getNeighbors(trainingSet, trainingLabels, instance, k):
    distances = []
    for x in range(len(trainingSet)):
        # Since the distance function is not symmetric, taking the distance in both directions
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingLabels[x], dist))
    # sorting by distance in ascending order
    distances.sort(key=operator.itemgetter(1))
    neighbors = [d[0] for d in distances[:k]]
    return neighbors

# k-NN logic to find the nearest neighbour's class
def nearestClass(neighbors):
    classVote = {}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1 
        else:
            classVote[response] = 1
    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from feature_extraction import loadDataset
from knn import getAccuracy, nearestClass, getNeighbors

data, labels = loadDataset(filename="lr.dat")

# Stratified K-Fold cross-validation with 10 splits
skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)
fold = 0
acc = 0
acc_list = []
# Performing cross-validation
for train_index, test_index in skf.split(data, labels):
    leng = len(test_index)
    predictions = []
    test_labels = []
    fold += 1
    data_train, data_test = data[train_index], data[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]   
    for x in range(leng):
        preds = nearestClass(getNeighbors(data_train, labels_train, data_test[x], 5))
        predictions.append(preds)
        test_labels.append(labels_test)    
    print(confusion_matrix(labels_test, predictions))
    accuracy1 = getAccuracy(labels_test, predictions)
    acc_list.append(accuracy1)
    print("Accuracy in fold "+ str(fold) + ": " + str(accuracy1 * 100))
print("Average accuracy: " + str(sum(acc_list) * 10))

[[ 8  0  1  0  0  0  0  0  1  0]
 [ 0 10  0  0  0  0  0  0  0  0]
 [ 0  0  6  0  0  0  0  0  1  3]
 [ 0  0  0  4  1  0  1  1  0  3]
 [ 0  0  0  0  8  0  1  1  0  0]
 [ 0  2  0  0  0  7  0  0  1  0]
 [ 0  0  0  1  0  0  8  0  0  1]
 [ 0  0  0  0  0  0  0 10  0  0]
 [ 0  0  1  1  3  0  0  0  5  0]
 [ 0  0  0  4  0  0  0  0  0  6]]
Accuracy in fold 1: 72.0
[[ 7  0  1  1  0  0  0  0  0  1]
 [ 0  8  0  0  0  2  0  0  0  0]
 [ 0  0  6  1  0  0  0  1  2  0]
 [ 0  0  0  9  0  0  0  0  1  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  1  0  0  0  9  0  0  0  0]
 [ 0  0  0  0  0  0  9  0  0  1]
 [ 0  0  0  0  0  0  0  9  1  0]
 [ 0  0  0  1  2  0  0  0  7  0]
 [ 0  0  0  4  0  0  0  0  0  6]]
Accuracy in fold 2: 80.0
[[ 8  0  0  2  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0]
 [ 0  0  8  1  0  0  1  0  0  0]
 [ 0  1  0  7  1  0  0  0  0  1]
 [ 0  0  0  0  8  0  0  1  1  0]
 [ 0  1  0  0  0  9  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0]
 [ 0  0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  2  0  0  